In [1]:
import json
import requests
import sys
from datetime import datetime, time, timedelta
from selenium import webdriver
GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')
from virasana.integracao.due import raspa_due

VIRASANA_URL = "https://ajna.labin.rf08.srf/virasana/"

2019-11-07 15:53:37,609 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../commons\ajna_commons\flask\access.log


In [2]:
diaapesquisar = datetime.today() - timedelta(days=4)
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2019-11-03  00:00:00 2019-11-03 23:59:59


In [13]:
conteineres_ids = raspa_due.raspa_containers_sem_due(datainicial, datafinal, None)


Conectando virasana
106 Contêineres recuperados.


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



In [14]:
def do_update(driver, tipo_manifesto):
    conteineres_ids = raspa_due.raspa_containers_sem_due(
        datainicial, datafinal, tipo_manifesto)
    print(len(conteineres_ids))
    try:
        raspa_due.auth_suite_rfb(driver)
        conteineres_listadue = raspa_due.get_dues_pos_acd(driver, list(conteineres_ids.keys()))
        due_detalhe = raspa_due.detalha_dues(driver, conteineres_listadue)
    finally:
        driver.close()
    print(len(conteineres_listadue))
    print(len(due_detalhe))
    pacote_carregamento = {}
    for conteiner, numeros_dues in conteineres_listadue.items():
        _id = conteineres_ids[conteiner]
        lista_dues = []
        for due in numeros_dues:
            if due is not None:
                pacote = raspa_due.monta_due_ajna(due_detalhe[due])
                lista_dues.append({'numero': due, **pacote})
        if numeros_dues and len(numeros_dues) > 0:
            pacote_carregamento[_id] = lista_dues
    r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento, verify=False)
    print(r.status_code)
    print(r.text)

In [15]:
driver = webdriver.Chrome(GECKO_PATH)

for tipo_manifesto in (None, 'lce'):
    do_update(driver, tipo_manifesto)

Conectando virasana
106 Contêineres recuperados.
97


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



97
1
201
{
  "status": "DUEs inseridas/atualizadas"
}

Conectando virasana


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



223 Contêineres recuperados.
221


WebDriverException: Message: invalid session id
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)
